# ~

## Set-up

### Imports

In [9]:
# Imports - standard
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests


In [2]:
# Imports - custom
import sys
sys.path.append(f"../code")
from paths import PROJECT_PATH, DATASET_PATH
from info import PATIENTS, FS, MATERIALS
from settings import *

In [3]:
# auto reload
%load_ext autoreload
%autoreload 2

### Settings

In [4]:
# 

### Functions

## Main

### Task-modulated electrodes

In [9]:
# load results of step 3 and merge with electrode info
fname = f"{PROJECT_PATH}/data/results/band_power_statistics.csv"
temp = pd.read_csv(fname, index_col=0)
temp = temp.loc[temp['memory']=='hit']
df_w = temp.loc[temp['material']=='words']
df_f = temp.loc[temp['material']=='faces']
for df in [df_w, df_f]: # compute joint significance
    df['sig_all'] = df[[f'{band}_sig' for band in BANDS]].all(axis=1)
    df['sig_any'] = df[[f'{band}_sig' for band in BANDS]].any(axis=1)


C:\Users\micha\AppData\Local\Temp\ipykernel_119884\3631205475.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sig_all'] = df[[f'{band}_sig' for band in BANDS]].all(axis=1)
C:\Users\micha\AppData\Local\Temp\ipykernel_119884\3631205475.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sig_any'] = df[[f'{band}_sig' for band in BANDS]].any(axis=1)
C:\Users\micha\AppData\Local\Temp\ipykernel_119884\3631205475.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Dat

(104, 97)

In [15]:
# print number (and percent) of significant channels for each band and both bands for each material

for df, material in zip([df_w, df_f], MATERIALS):
    print(material)
    for band in BANDS:
        print(f"{band}: {df[band+'_sig'].sum()} ({df[band+'_sig'].mean()*100:.1f}%)")
    print(f"joint: {df['sig_all'].sum()} ({df['sig_all'].mean()*100:.1f}%)")
    print()

words
alpha: 248 (35.7%)
gamma: 238 (34.2%)
joint: 104 (15.0%)

faces
alpha: 235 (33.8%)
gamma: 221 (31.8%)
joint: 97 (14.0%)



### SpecParam r-squared

In [17]:
from specparam import SpectralGroupModel
from specparam_utils import compute_adj_r2


In [18]:
r2_knee = []
r2_fixed = []

# loop through conditions
for material in ['words','faces']:
    for memory in ['hit','miss']:
        for window in ['pre', 'post']:
            # load r-squared values for 'knee' model
            fg_k = SpectralGroupModel()
            fname = f"psd_{material}_{memory}_{window}stim_params_knee"
            fg_k.load(f'{PROJECT_PATH}/data/ieeg_psd_param/{fname}')
            r2_k = compute_adj_r2(fg_k)
            r2_knee.append(r2_k)

            # load r-squared values for 'fixed' model
            fg_f = SpectralGroupModel()
            fname = f"psd_{material}_{memory}_{window}stim_params_fixed"
            fg_f.load(f'{PROJECT_PATH}/data/ieeg_psd_param/{fname}')
            r2_f = compute_adj_r2(fg_f)
            r2_fixed.append(r2_f)



In [22]:
print(f"Fixed model: {np.nanmean(r2_fixed):.3f}")
print(f"Knee model: {np.nanmean(r2_knee):.3f}")

Fixed model: 0.960
Knee model: 0.976


### Hierarchical bootstrap

In [23]:
fname = r"C:\Users\micha\projects\oscillation_vs_exponent\data\ieeg_stats\group_level_hierarchical_bootstrap_active_bymaterial.csv"
df = pd.read_csv(fname, index_col=0)
df = df.loc[df['memory']=='hit']
df

,material,memory,feature,pvalue,cohens_d
0,faces,hit,exponent,0.000,0.439738
1,faces,hit,alpha,0.006,0.173087
2,faces,hit,alpha_adj,0.040,0.207368
3,faces,hit,gamma,0.003,-0.141586
4,faces,hit,gamma_adj,0.273,-0.131028
10,words,hit,exponent,0.000,0.358535
11,words,hit,alpha,0.004,0.143657
12,words,hit,alpha_adj,0.002,0.255376
13,words,hit,gamma,0.003,-0.089741
14,words,hit,gamma_adj,0.146,-0.285829


In [25]:
# correct for multiple comparisons
from statsmodels.stats.multitest import multipletests
pvalues = df['pvalue']
pvalues[pvalues==0] = 0.001
stats = multipletests(pvalues, alpha=0.05, method='holm')
df['p_corr'] = stats[1]
df['sig'] = df['p_corr'] < 0.05
df

,material,memory,feature,pvalue,cohens_d,p_corr,sig
0,faces,hit,exponent,0.001,0.439738,0.010,True
1,faces,hit,alpha,0.006,0.173087,0.024,True
2,faces,hit,alpha_adj,0.040,0.207368,0.120,False
3,faces,hit,gamma,0.003,-0.141586,0.021,True
4,faces,hit,gamma_adj,0.273,-0.131028,0.292,False
10,words,hit,exponent,0.001,0.358535,0.010,True
11,words,hit,alpha,0.004,0.143657,0.021,True
12,words,hit,alpha_adj,0.002,0.255376,0.016,True
13,words,hit,gamma,0.003,-0.089741,0.021,True
14,words,hit,gamma_adj,0.146,-0.285829,0.292,False


### Intersection Frequency

In [37]:
data_in = np.load(f"{PROJECT_PATH}/data/ieeg_intersection_results/intersection_results_faces_hit_knee.npz")
intersection_f = data_in['intersection'][df_f['sig_all'].values]
data_in = np.load(f"{PROJECT_PATH}/data/ieeg_intersection_results/intersection_results_words_hit_knee.npz")
intersection_w = data_in['intersection'][df_w['sig_all'].values]

# print median and std
print(f"Median intersection:")
print(f"    Words:\n\tmedian: {np.nanmedian(intersection_w):.3f}\n\tstd: {np.nanstd(intersection_w):.3f}")
print(f"    Faces:\n\tmedian: {np.nanmedian(intersection_f):.3f}\n\tstd: {np.nanstd(intersection_f):.3f}")

Median intersection:
    Words:
	median: 32.936
	std: 20.863
    Faces:
	median: 37.926
	std: 20.803


### Regression of confounding variables

---------------------------------------  
Word-encoding  
---------------------------------------  
alpha results:  
        R-squared: 0.177  
        F-statistic: 287.733  
        p-value: 1.356e-58  

alpha_adj results:  
        R-squared: 0.003  
        F-statistic: 4.434  
        p-value: 0.035  

gamma results:  
        R-squared: 0.229  
        F-statistic: 396.464  
        p-value: 1.887e-77

gamma_adj results:  
        R-squared: 0.001  
        F-statistic: 1.531  
        p-value: 0.216  


alpha cross-validation t-test results:  
        T-statistic: 4.013  
        p-value: 0.002  

gamma cross-validation t-test results:  
        T-statistic: 3.315  
        p-value: 0.006  

---------------------------------------  
Face-encoding  
---------------------------------------    
alpha results:  
        R-squared: 0.131  
        F-statistic: 99.716  
        p-value: 5.716e-22  

alpha_adj results:  
        R-squared: 0.001  
        F-statistic: 0.656  
        p-value: 0.418  

gamma results:  
        R-squared: 0.174  
        F-statistic: 139.814  
        p-value: 2.087e-29  

gamma_adj results:  
        R-squared: 0.005  
        F-statistic: 3.403  
        p-value: 0.066  

alpha cross-validation t-test results:  
        T-statistic: 3.405  
        p-value: 0.005  

gamma cross-validation t-test results:  
        T-statistic: 1.788  
        p-value: 0.099  


### Behavioral performance

In [40]:
from mne import read_epochs

# load behavioral data for all patients
dir_input = f"{PROJECT_PATH}/data/ieeg_epochs/"
files = os.listdir(dir_input)
df_list = []
for fname in files:
    f_parts = fname.split('_')
    epochs = read_epochs(f"{dir_input}/{fname}", verbose=False)
    df_i = epochs.metadata
    df_i.insert(0, 'patient', f_parts[0])
    df_i.insert(1, 'material', f_parts[1])
    df_list.append(df_i)
df = pd.concat(df_list)
df

,patient,material,trial_num,pleasantness,confidence,recalled,reaction_time
0,pat02,faces,2.0,4.0,3.0,1.0,1780.1
2,pat02,faces,4.0,2.0,1.0,1.0,2208.4
3,pat02,faces,5.0,2.0,2.0,1.0,2369.6
4,pat02,faces,6.0,1.0,2.0,1.0,2301.8
5,pat02,faces,7.0,3.0,1.0,1.0,2365.2
...,...,...,...,...,...,...,...
83,pat22,words,86.0,3.0,4.0,0.0,4964.1
85,pat22,words,88.0,3.0,4.0,0.0,5109.9
92,pat22,words,95.0,5.0,5.0,0.0,3632.1
93,pat22,words,96.0,5.0,4.0,0.0,4930.6


In [41]:
# count number of trials and hits per patient
behav_list = [] 
patients = df['patient'].unique()
for material in ['faces', 'words']:
    df_i = df.loc[df['material']==material]
    n_trials = df_i.groupby('patient').size().values
    n_correct = df_i.groupby('patient').sum()['recalled'].values.astype(int)
    behav_i = pd.DataFrame({'patient': patients, 'material': material,
        'n_trials': n_trials, 'n_correct': n_correct})
    behav_list.append(behav_i)
behav = pd.concat(behav_list)
behav.insert(4, 'pct_correct', behav['n_correct']/behav['n_trials'])
behav

,patient,material,n_trials,n_correct,pct_correct
0,pat02,faces,99,63,0.636364
1,pat04,faces,99,60,0.606061
2,pat05,faces,97,65,0.670103
3,pat08,faces,94,56,0.595745
4,pat10,faces,100,66,0.660000
5,pat11,faces,100,44,0.440000
6,pat15,faces,100,49,0.490000
7,pat16,faces,100,21,0.210000
8,pat17,faces,100,35,0.350000
9,pat19,faces,100,51,0.510000


In [45]:
# print mean accuracy for each block
print(f"Overall mean: {behav['pct_correct'].mean()}")
print(behav.iloc[:,1:].groupby('material').mean()['pct_correct'])

Overall mean: 0.6136163225042365
material
faces    0.529867
words    0.697366
Name: pct_correct, dtype: float64


### single-channel stats

In [4]:
FEATURES = ['exponent', 'alpha', 'alpha_adj', 'gamma', 'gamma_adj']


In [5]:

def load_stats():
    # load permutatiobn results
    fname = f"{PROJECT_PATH}/data/results/band_power_statistics.csv"
    df_stats = pd.read_csv(fname, index_col=0)
    df_stats = df_stats.loc[df_stats['memory']=='hit']

    # find channels with significant alpha and gamma modulation
    df_stats['sig_all'] = df_stats['alpha_sig'] & df_stats['gamma_sig']
    df_stats['sig_any'] = df_stats['alpha_sig'] | df_stats['gamma_sig']

     #split by conditoin
    df_w = df_stats.loc[df_stats['material']=='words']
    df_f = df_stats.loc[df_stats['material']=='faces']
    
    return df_w, df_f


In [6]:
df_w, df_f = load_stats()

for df in [df_w, df_f]:
    df.drop(columns=['alpha_pre', 'alpha_post', 'alpha_pval', 'alpha_sign', 
                     'gamma_pre', 'gamma_post', 'gamma_pval', 'gamma_sign',
                     'alpha_sig', 'gamma_sig', 'sig_any'], inplace=True)
    
df_w

,patient,material,memory,chan_idx,sig_all
18,pat02,words,hit,0,False
19,pat02,words,hit,1,False
20,pat02,words,hit,2,False
21,pat02,words,hit,3,False
22,pat02,words,hit,4,False
...,...,...,...,...,...
2722,pat22,words,hit,48,False
2723,pat22,words,hit,49,False
2724,pat22,words,hit,50,False
2725,pat22,words,hit,51,True


In [7]:
# load
df_wst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_words.csv", index_col=0)
df_fst = pd.read_csv(f"{PROJECT_PATH}/data/results/single_channel_stats_faces.csv", index_col=0)

# merge
df_w = df_wst.merge(df_w, on=['patient','chan_idx'])
df_f = df_fst.merge(df_f, on=['patient','chan_idx'])

# drop non-task-modulated channels
df_w = df_w.loc[df_w['sig_all']].drop(columns=['sig_all'])
df_f = df_f.loc[df_f['sig_all']].drop(columns=['sig_all'])

# show
print(len(df_w), len(df_f))
df_w

104 97


,index,patient,chan_idx,label,pos_y,pos_x,pos_z,exponent_t,exponent_p,alpha_t,alpha_p,alpha_adj_t,alpha_adj_p,gamma_t,gamma_p,gamma_adj_t,gamma_adj_p,material,memory
9,100,pat04,0,A01-A09,-58.30120,52.43075,-26.23685,4.032624,2.107200e-04,6.770440,2.233319e-08,0.671490,0.505341,-8.912848,1.682913e-11,-1.907723,0.062821,words,hit
54,145,pat04,45,A01-A02,-63.34330,53.12490,-21.48615,5.868628,4.555194e-07,7.522039,1.510788e-09,0.168096,0.867245,-6.706357,2.515065e-08,-0.123916,0.901921,words,hit
58,149,pat04,49,A10-A11,-52.12025,61.31445,-15.30360,0.523225,6.031756e-01,1.630562,1.093948e-01,2.450568,0.017880,-1.450528,1.532829e-01,0.796636,0.429505,words,hit
67,158,pat04,58,A28-A29,-26.97260,66.43845,5.20775,-1.827064,7.332320e-02,1.439572,1.558691e-01,0.685729,0.495873,2.614423,1.161130e-02,0.534061,0.595532,words,hit
88,179,pat04,79,A60-A62,16.19335,56.90075,1.58265,1.386216,1.723639e-01,1.377683,1.749672e-01,-0.580096,0.564681,1.428293,1.599625e-01,-0.306261,0.760788,words,hit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686,1381,pat22,44,LOF_08-LOF_09,47.00000,-25.00000,24.00000,1.914847,6.062523e-02,1.336771,1.867030e-01,0.655598,0.514767,-4.709344,1.679407e-05,1.287626,0.203172,words,hit
688,1383,pat22,46,MOF_01-MOF_02,28.00000,-14.00000,-11.50000,0.060266,9.521586e-01,-0.811777,4.203569e-01,2.296161,0.025431,1.534736,1.304796e-01,-0.454518,0.651213,words,hit
689,1384,pat22,47,MOF_02-MOF_03,31.50000,-14.00000,-4.00000,1.026219,3.092008e-01,-1.514964,1.354059e-01,0.288600,0.773953,2.564863,1.302896e-02,1.182913,0.241841,words,hit
693,1388,pat22,51,MOF_08-MOF_09,48.00000,-14.00000,23.50000,3.753384,4.164932e-04,0.383175,7.030407e-01,0.223396,0.824040,-6.713844,1.026949e-08,-0.146502,0.884052,words,hit


In [10]:
# compute corrected pvalues - task-modulated channels

for df_sig, material in zip([df_w, df_f], ['words', 'faces']):
    print(f"\n\n{material}-encoding ({len(df_sig)} channels):")
    # correct p-values
    for feature in FEATURES:
        results = multipletests(df_sig[f"{feature}_p"], alpha=0.05, method='holm')
        df_sig.insert(df_sig.shape[1], f"{feature}_pval_corr", results[1])

    # determine significance and direction of effect
    for feature in FEATURES:
        df_sig.insert(df_sig.shape[1], f'{feature}_sig_corr', 0)
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']>0), f'{feature}_sig_corr'] = -1 
        df_sig.loc[(df_sig[f'{feature}_pval_corr']<0.05) & (df_sig[f'{feature}_t']<0), f'{feature}_sig_corr'] = 1 

    # print number of significant channels
    for feature in FEATURES:
        print(f"\n{feature}:")
        print(f"\tIncrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tDecrease: {df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==-1].shape[0]/df_sig.shape[0]*100:.2f}%)")
        print(f"\tNo change: {df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]} ({df_sig.loc[df_sig[f'{feature}_sig_corr']==0].shape[0]/df_sig.shape[0]*100:.2f}%)")



words-encoding (104 channels):

exponent:
	Increase: 0 (0.00%)
	Decrease: 22 (21.15%)
	No change: 82 (78.85%)

alpha:
	Increase: 6 (5.77%)
	Decrease: 33 (31.73%)
	No change: 65 (62.50%)

alpha_adj:
	Increase: 0 (0.00%)
	Decrease: 5 (4.81%)
	No change: 99 (95.19%)

gamma:
	Increase: 37 (35.58%)
	Decrease: 4 (3.85%)
	No change: 63 (60.58%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 104 (100.00%)


faces-encoding (97 channels):

exponent:
	Increase: 2 (2.06%)
	Decrease: 26 (26.80%)
	No change: 69 (71.13%)

alpha:
	Increase: 9 (9.28%)
	Decrease: 30 (30.93%)
	No change: 58 (59.79%)

alpha_adj:
	Increase: 3 (3.09%)
	Decrease: 4 (4.12%)
	No change: 90 (92.78%)

gamma:
	Increase: 39 (40.21%)
	Decrease: 5 (5.15%)
	No change: 53 (54.64%)

gamma_adj:
	Increase: 0 (0.00%)
	Decrease: 0 (0.00%)
	No change: 97 (100.00%)
